In [1]:
import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import pickle
import numpy

/home/fmpaezri/.local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")
ERROR:root:Problem importing Lasagne: requires GPU support -- see http://lasagne.readthedocs.org/en/latest/user/installation.html#gpu-support


In [52]:
from PIL import Image
import IPython.display

In [15]:
import tracking.VideoSequence as vs

In [2]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [3]:
with open('/home/fmpaezri/repos/localization-agent/model.pkl', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

ERROR:root:Problem importing Lasagne: requires GPU support -- see http://lasagne.readthedocs.org/en/latest/user/installation.html#gpu-support


CPU times: user 17.7 s, sys: 1.6 s, total: 19.3 s
Wall time: 27.9 s


In [4]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [76]:
tester = tracking.Tester.Tester(tracker)

In [77]:
data, labels = tester.loadImageSequence('/home/fmpaezri/datasets/vot-challenge/sequences2014/ball/', newShape=(rnn.imgSize,rnn.imgSize))

In [81]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [82]:
labels = labels[numpy.newaxis,...]

In [83]:
print data.shape, labels.shape

(1, 603, 100, 100, 3) (1, 603, 4)


In [84]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, False, labels.shape[1], 4, '/tmp')

CPU times: user 29 s, sys: 1.94 s, total: 30.9 s
Wall time: 16.9 s


In [85]:
predLabels*rnn.imgSize

array([[[-100.        ,  -14.29862678,   51.96136236,    5.7856217 ],
        [-100.        ,  -20.61040699,   24.27529991,   13.33328038],
        [-100.        ,  -25.4465878 ,   17.32102782,   13.11986446],
        ..., 
        [-100.        ,  -34.90288854,   17.73682982,    4.56309542],
        [-100.        ,  -34.90513563,   17.73575544,    4.55895327],
        [-100.        ,  -34.91114676,   17.74177402,    4.55427654]]])

In [86]:
predLabels.shape

(1, 603, 4)

In [87]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [88]:
seq.addBoxes(labels[0], 'green')#/[320, 240, 320, 240]*rnn.imgSize
seq.addBoxes((predLabels[0]+1)*rnn.imgSize/2, 'red')

In [89]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [90]:
IPython.display.HTML(vs.displayHTML(outputName))

In [75]:
reload(tracking.Tester)

<module 'tracking.Tester' from '/home/fmpaezri/repos/localization-agent/tracking/Tester.py'>

In [93]:
iou = tracking.Tester.getIntOverUnion(labels, (predLabels+1)*rnn.imgSize/2)

In [94]:
iou

array([[-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.

In [95]:
labels

array([[[ 62.609375  ,  66.38333333,  76.7125    ,  47.39166667],
        [ 63.246875  ,  66.38333333,  77.209375  ,  47.39166667],
        [ 63.53125   ,  66.19583333,  77.49375   ,  47.39166667],
        ..., 
        [ 42.946875  ,  84.14166667,  54.771875  ,  68.09166667],
        [ 42.30625   ,  84.2375    ,  54.275     ,  68.27916667],
        [ 42.30625   ,  84.2375    ,  54.275     ,  68.27916667]]])

In [96]:
(predLabels[0]+1)*rnn.imgSize/2

array([[  0.        ,  42.85068661,  75.98068118,  52.89281085],
       [  0.        ,  39.6947965 ,  62.13764995,  56.66664019],
       [  0.        ,  37.2767061 ,  58.66051391,  56.55993223],
       ..., 
       [  0.        ,  32.54855573,  58.86841491,  52.28154771],
       [  0.        ,  32.54743218,  58.86787772,  52.27947664],
       [  0.        ,  32.54442662,  58.87088701,  52.27713827]])